## Feature selection


In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
import pyarrow.parquet as pq
import pyarrow as pa

In [3]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [4]:
train_file = r"amex\agg_train_all_rev.parquet"

In [5]:
%%time
df=pd.read_parquet(train_file, engine='pyarrow')


Wall time: 4.6 s


In [6]:
df.head(2)

,customer_ID,D_104,D_73,D_77,R_24,D_145,D_123,B_31=0,B_31=1,D_91,B_2,D_135,R_10,S_20,R_21,D_112,D_124,D_63=CL,D_63=CO,D_63=CR,D_63=XL,D_63=XM,D_63=XZ,S_11,D_125,D_46,R_1,R_6,B_3,R_12,D_106,B_18,S_15,S_12,B_6,B_22,D_41,D_103,D_108,B_39,S_16,D_120=0.0,D_120=1.0,S_23,S_2=min,S_2=max,days,S_18,R_2,D_131,R_25,B_5,B_25,D_52,D_53,D_89,D_144,D_78,P_2,D_56,D_119,P_3,B_16,S_19,B_15,D_137,D_122,D_141,D_142,D_43,B_9,D_45,S_24,D_58,S_7,R_22,D_143,B_30=0.0,B_30=1.0,B_30=2.0,B_27,D_66=0.0,D_66=1.0,B_36,D_139,D_118,D_114=0.0,D_114=1.0,B_4,B_24,D_102,S_6,D_136,B_33,D_138,D_105,R_3,B_11,D_47,D_59,B_1,D_60,S_5,B_41,B_14,D_50,R_7,B_8,D_134,D_132,D_127,R_4,D_65,B_23,B_42,D_74,D_79,D_130,B_19,D_68=0.0,D_68=1.0,D_68=2.0,D_68=3.0,D_68=4.0,D_68=5.0,D_68=6.0,S_8,R_17,P_4,D_81,D_128,B_37,R_15,D_70,D_71,B_32,S_9,D_93,R_8,S_22,D_61,B_7,D_86,B_26,B_20,S_13,D_126=-1.0,D_126=0.0,D_126=1.0,D_69,R_16,cnt,D_109,D_64=-1,D_64=O,D_64=R,D_64=U,D_113,R_23,R_13,B_29,D_133,D_49,B_17,D_39,D_87,D_117=-1.0,D_117=1.0,D_117=2.0,D_117=3.0,D_117=4.0,D_117=5.0,D_117=6.0,D_140,S_3,D_62,D_72,R_19,D_80,D_54,B_38=1.0,B_38=2.0,B_38=3.0,B_38=4.0,B_38=5.0,B_38=6.0,B_38=7.0,D_84,D_116=0.0,D_116=1.0,D_75,D_83,D_111,D_121,D_110,R_26,R_20,D_82,B_13,R_18,D_88,S_26,B_40,R_14,B_21,D_48,R_11,S_17,R_28,R_9,D_92,S_27,D_76,D_96,D_115,D_42,R_5,B_10,D_94,R_27,B_12,D_55,D_107,D_44,S_25,D_51,B_28,D_129,log_B_11,log_B_12,log_B_13,log_B_21,log_B_22,log_B_23,log_B_24,log_B_26,log_B_27,log_B_28,log_B_29,log_B_3,log_B_32,log_B_36,log_B_4,log_B_40,log_B_41,log_B_42,log_B_5,log_B_9,log_D_106,log_D_107,log_D_108,log_D_109,log_D_113,log_D_115,log_D_118,log_D_119,log_D_123,log_D_125,log_D_131,log_D_133,log_D_135,log_D_136,log_D_137,log_D_138,log_D_140,log_D_39,log_D_41,log_D_43,log_D_44,log_D_45,log_D_49,log_D_51,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.009606,0.163683,0.419295,0.005469,0.005814,0.003103,0.0,13.0,1.197627,1.005086,0.030999,0.004742,0.006292,0.004931,1.004348,0.686641,0.0,0.0,13.0,0.0,0.0,0.0,0.469887,0.006253,0.378074,0.004509,0.006067,0.006456,1.004889,0.19425,0.842565,0.173335,0.247988,0.11351,0.005700,0.005021,1.005188,0.056857,0.213355,0.004974,13.0,0.0,0.135036,2017-03-09,2018-03-13,369,0.005075,0.005754,0.005446,0.005799,0.146650,0.005915,0.204972,0.078497,0.005235,0.005283,0.005218,0.933824,0.158571,0.244733,0.680138,0.004769,0.005955,0.026247,0.017057,0.433732,0.005178,0.382473,0.153943,0.006220,0.725369,0.931956,0.064803,0.098374,0.005002,0.005066,13.0,0.0,0.0,0.004098,0.0,0.0,0.005292,0.003664,0.245514,0.0,13.0,0.040309,0.005135,0.856909,0.004647,0.212771,1.004033,0.16938,0.984966,0.004752,0.007230,0.532874,0.061147,0.012007,0.534817,0.029112,0.006753,0.023142,0.150326,0.005851,0.003585,0.334751,0.185971,1.006123,0.006656,0.004701,0.026177,0.104542,0.039496,0.004170,0.005195,0.005231,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.797145,0.004164,0.005670,0.005834,1.004154,0.012015,0.005171,0.005221,0.214785,0.005084,0.039818,0.004415,0.005595,0.916144,0.225847,0.036624,0.004874,0.004408,0.005731,0.654700,0.0,0.0,13.0,0.005275,0.004740,13,0.006114,0.0,13.0,0.0,0.0,0.005476,0.004593,0.004522,0.029967,0.004760,0.181014,0.680789,0.010704,1.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.005343,0.113215,0.181286,0.004495,0.005231,0.081974,1.006225,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.004945,13.0,0.0,0.036125,0.004892,0.868921,0.711829,0.740351,0.085475,0.005231,0.505637,0.100432,0.006589,0.180269,0.012560,0.087300,0.005879,0.005500,0.240978,0.004784,0.005079,0.004089,0.211189,0.543890,0.850951,0.138271,0.003619,0.247095,0.180571,0.004684,0.270280,0.005909,1.005594,0.125683,0.224432,0.672166,0.004673,0.974669,0.978914,0.050968,1.005537,-4.929449,-2.073995,-2.298275,-5.203047,-5.167329,-3.642872,-5.271716,-5.424415,-5.497164,-2.976560,-5.223177,-5.042712,-5.281577,-5.241639,-3.211186,-2.438409,-4.997762,-3.590491,-1.919705,-5.079943,-2.28975,-0.397250,-5.240566,-5.097225,-5.207382,-1.397983,-1.404402,-1.407586,-5.775453,-5.074720,-5.212901,-5.347446,-4.912965,-2.413825,-5.17097,-3.796226,-5.232025,-4.537

### feature selection

In [33]:
all_feats = df.columns.tolist()
len(all_feats)
x_cols = list(set(all_feats)-set(['S_2=max', 'S_2=min'] + ['customer_ID', 'target']))
print(len(x_cols))

269


In [34]:
print(x_cols)

['B_28', 'D_82', 'D_113', 'D_88', 'B_36', 'log_D_107', 'log_B_26', 'D_114=1.0', 'D_68=6.0', 'B_4', 'B_30=0.0', 'log_B_13', 'R_18', 'S_9', 'D_68=2.0', 'D_61', 'B_38=4.0', 'D_65', 'S_8', 'D_106', 'B_5', 'R_11', 'D_75', 'log_B_32', 'S_12', 'R_15', 'log_B_24', 'R_5', 'log_D_137', 'B_6', 'log_B_22', 'D_62', 'R_4', 'D_142', 'D_108', 'D_126=1.0', 'S_5', 'log_D_138', 'D_43', 'log_B_29', 'D_107', 'log_B_40', 'B_20', 'log_B_9', 'B_26', 'D_79', 'D_77', 'B_37', 'S_7', 'B_11', 'S_11', 'R_16', 'R_10', 'D_134', 'D_120=0.0', 'D_105', 'R_22', 'D_78', 'log_B_42', 'R_24', 'D_86', 'log_D_51', 'B_23', 'B_39', 'R_7', 'log_D_115', 'D_118', 'D_54', 'log_B_4', 'S_26', 'B_22', 'log_D_119', 'D_143', 'S_16', 'R_8', 'R_25', 'B_18', 'D_129', 'B_27', 'B_12', 'D_145', 'D_63=CL', 'D_138', 'R_9', 'D_126=0.0', 'log_D_39', 'S_23', 'D_120=1.0', 'D_130', 'B_14', 'P_3', 'S_24', 'R_26', 'S_13', 'D_91', 'D_73', 'D_102', 'D_70', 'R_23', 'B_10', 'log_B_12', 'log_D_118', 'D_68=5.0', 'D_52', 'D_117=4.0', 'D_136', 'D_49', 'log_B_3

In [35]:

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from spFSR import SpFSR


In [36]:
x=df[x_cols]
y=df['target']
x.shape, y.shape

((458913, 269), (458913,))

In [37]:
x.head(2)

,B_28,D_82,D_113,D_88,B_36,log_D_107,log_B_26,D_114=1.0,D_68=6.0,B_4,B_30=0.0,log_B_13,R_18,S_9,D_68=2.0,D_61,B_38=4.0,D_65,S_8,D_106,B_5,R_11,D_75,log_B_32,S_12,R_15,log_B_24,R_5,log_D_137,B_6,log_B_22,D_62,R_4,D_142,D_108,D_126=1.0,S_5,log_D_138,D_43,log_B_29,D_107,log_B_40,B_20,log_B_9,B_26,D_79,D_77,B_37,S_7,B_11,S_11,R_16,R_10,D_134,D_120=0.0,D_105,R_22,D_78,log_B_42,R_24,D_86,log_D_51,B_23,B_39,R_7,log_D_115,D_118,D_54,log_B_4,S_26,B_22,log_D_119,D_143,S_16,R_8,R_25,B_18,D_129,B_27,B_12,D_145,D_63=CL,D_138,R_9,D_126=0.0,log_D_39,S_23,D_120=1.0,D_130,B_14,P_3,S_24,R_26,S_13,D_91,D_73,D_102,D_70,R_23,B_10,log_B_12,log_D_118,D_68=5.0,D_52,D_117=4.0,D_136,D_49,log_B_36,D_123,R_6,B_38=7.0,S_3,log_D_109,B_33,D_117=6.0,D_41,B_30=1.0,D_64=O,log_D_123,D_63=XZ,D_63=CO,cnt,log_B_23,D_141,D_84,D_104,S_22,D_60,D_71,D_139,log_B_27,R_21,D_114=0.0,D_115,R_19,D_69,R_17,B_30=2.0,D_140,R_27,D_66=1.0,D_110,D_55,log_B_28,D_135,D_68=1.0,B_38=6.0,D_111,R_3,D_117=1.0,D_125,D_81,log_D_140,S_6,D_80,log_B_11,R_1,log_D_108,log_D_131,log_D_49,log_D_113,B_15,B_1,R_14,D_48,S_25,P_2,days,log_B_21,D_93,log_D_135,B_19,D_132,R_28,D_128,D_103,D_63=XM,B_7,D_127,R_20,D_117=5.0,B_38=5.0,D_58,B_38=3.0,D_39,B_17,D_96,D_137,D_68=4.0,B_9,B_31=0,D_56,R_2,D_116=0.0,S_19,P_4,D_109,D_45,log_B_41,D_47,log_B_5,D_94,S_18,D_112,B_8,D_122,log_D_106,D_144,D_74,D_46,log_D_133,B_38=2.0,D_117=-1.0,D_63=XL,S_17,log_B_3,B_42,D_117=3.0,D_44,D_124,log_D_41,D_63=CR,log_D_136,B_21,D_131,D_126=-1.0,D_92,R_12,D_42,D_76,B_16,D_50,B_3,S_27,D_72,D_64=U,D_51,D_133,S_15,S_20,log_D_125,D_68=0.0,D_119,D_64=R,R_13,D_59,B_24,D_53,B_41,B_13,log_D_43,D_89,log_D_45,D_83,B_29,B_2,D_116=1.0,D_68=3.0,D_117=2.0,B_25,D_66=0.0,B_38=1.0,log_D_44,D_121,B_32,D_64=-1,B_40,B_31=1,D_87
0,0.050968,0.505637,0.005476,0.180269,0.005292,-0.397250,-5.424415,13.0,13.0,0.040309,13.0,-2.298275,0.006589,0.039818,0.0,0.225847,0.0,0.004701,0.797145,0.19425,0.146650,0.004784,0.036125,-5.281577,0.247988,0.005171,-5.271716,0.004684,-5.17097,0.11351,-5.167329,0.181286,0.006656,0.382473,0.056857,13.0,0.029112,-3.796226,0.153943,-5.223177,0.672166,-2.438409,0.005731,-5.079943,0.004408,0.004170,0.419295,0.012015,0.098374,0.007230,0.469887,0.004740,0.004742,0.334751,13.0,0.984966,0.005002,0.005218,-3.590491,0.005469,0.004874,-0.021312,0.026177,0.213355,0.005851,-1.397983,0.245514,1.006225,-3.211186,0.012560,0.005700,-1.407586,0.005066,0.004974,0.005595,0.005799,0.842565,1.005537,0.004098,0.125683,0.005814,0.0,0.16938,0.211189,0.0,-4.537166,0.135036,0.0,0.005195,0.023142,0.680138,0.931956,0.085475,0.654700,1.197627,0.163683,0.856909,0.005221,0.004593,0.270280,-2.073995,-1.404402,0.0,0.204972,13.0,0.212771,0.181014,-5.241639,0.003103,0.006067,0.0,0.113215,-5.097225,1.004033,0.0,0.005021,0.0,13.0,-5.775453,0.0,0.0,13,-3.642872,0.005178,0.004945,1.009606,0.916144,0.534817,0.214785,0.003664,-5.497164,0.004931,0.0,0.247095,0.005231,0.005275,0.004164,0.0,0.005343,1.005594,0.0,0.740351,0.224432,-2.976560,0.030999,0.0,0.0,0.868921,0.004752,0.0,0.006253,0.005834,-5.232025,0.004647,0.081974,-4.929449,0.004509,-5.240566,-5.212901,-2.241847,-5.207382,0.026247,0.012007,0.005879,0.240978,0.974669,0.933824,369,-5.203047,0.004415,-4.912965,0.005231,0.185971,0.004089,1.004154,1.005188,0.0,0.036624,1.006123,0.005231,0.0,0.0,0.064803,0.0,0.010704,0.680789,0.003619,0.017057,0.0,0.006220,0.0,0.158571,0.005754,13.0,0.005955,0.005670,0.006114,0.725369,-4.997762,0.532874,-1.919705,0.005909,0.005075,1.004348,0.003585,0.433732,-2.28975,0.005283,0.039496,0.378074,-5.347446,13.0,0.0,0.0,0.005079,-5.042712,0.104542,0.0,0.004673,0.686641,-5.294216,13.0,-2.413825,0.005500,0.005446,0.0,0.543890,1.004889,0.180571,0.138271,0.004769,0.150326,0.006456,0.850951,0.004495,0.0,0.978914,0.004760,0.173335,0.006292,-5.074720,0.0,0.244733,0.0,0.004522,0.061147,0.005135,0.078497,0.006753,0.100432,-2.376637,0.005235,-0.321075,0.004892,0.029967,1.005086,0.0,0.0,0.0,0.005915,0.0,0.0,-5.365879,0.711829,0.005084,0.0,0.087300,13.0,1.0
1,0.017052,0.443825,0.006391,0.180269,0.006

In [38]:
y.head(2)

0    0
1    0
Name: target, dtype: int64

In [39]:
for pct in [0.35, 0.5, 0.75, 0.95]:
    print('{:,.0f}'.format(pct*x.shape[0])) # print("%.2f" % z)  print("{:.2f}".format(z))

160,620
229,456
344,185
435,967


In [ ]:
%%time
# set the engine parameters
#sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=1))

for n_samples in [150000, 250000, 350000, 435000]:
    
    print(n_samples, '='*100)

    sp_run = sp_engine.run(num_features=0, n_samples_max=n_samples, print_freq=1, iter_max=10)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)